In [1]:
import datasets
from datasets import load_dataset, load_from_disk
from in_context_ssl.reasoning.template import *
import os
import openai
from openai import OpenAI
from tqdm import tqdm
import numpy as np
from pydantic import BaseModel, Field
import json
from in_context_ssl.reasoning.utils import *
from in_context_ssl.reasoning.dataset import *
import re
import pandas as pd
from in_context_ssl.reasoning.utils import *
import torchmetrics
import matplotlib.pyplot as plt

/Users/zhengyaogu/miniconda3/envs/in-context-ssl/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# set your API here
os.environ["OPENAI_API_KEY"] = ""
client = OpenAI()

## Preprocessing

In [29]:
def create_translation_dataset(target_lang, stage):
    split = "dev" if stage == "train" else "devtest"
    ds = load_dataset("openlanguagedata/flores_plus")[split]
    df  = ds.to_pandas()

    def add_embedding(doc):
        out_doc = {
            "embedding": client.embeddings.create(
                input=[doc["question"]],
                model="text-embedding-3-large"
            ).data[0].embedding
        }
        return out_doc

    df_source = df[df["iso_639_3"] == "eng"]
    df_target = df[df["iso_639_3"] == target_lang]
    joined_df = pd.merge(df_source, df_target, on="id", how="inner")[[
        "text_x", "text_y", "topic_x"
    ]]
    joined_df = joined_df.rename(columns={
        "text_x": "question",
        "text_y": "answer",
        "topic_x": "group"
    })

    ds = datasets.Dataset.from_pandas(joined_df)
    ds = ds.map(add_embedding)
    ds.save_to_disk("in_context_ssl/reasoning/data/flores_{}_{}.hf".format(target_lang, stage))
    return ds


In [33]:
ds = load_dataset("openlanguagedata/flores_plus")["devtest"]
df  = ds.to_pandas()
df_source = df[df["iso_639_3"] == "bem"] # 

In [37]:
target_langs = ["bem"]
ds = load_from_disk("in_context_ssl/reasoning/data/flores_{}_test.hf".format(lang))
ds = ds.shuffle()
ds = ds.select(range(200))
ds.save_to_disk("in_context_ssl/reasoning/data/flores_{}_test_new.hf".format(lang))

Saving the dataset (1/1 shards): 100%|██████████| 200/200 [00:00<00:00, 3683.71 examples/s]


## Inference

In [32]:
k_total = 100
k_gt = 0

ds = TranslationDatasetSrd()
print(ds.get_demonstrations(
    "in_context_ssl/reasoning/data/flores_srd_psl_k={}_verbalized.hf".format(k_gt),
    k=k_total-k_gt, k_gt=k_gt, 
    style="psl", answer=True, rationale=False, quantile=0.9, topk=False, seed=42
))
preds = []
gold = []
messages = []

for inst in tqdm(ds):
    choice = query_openai(client, inst["query"], model="gpt-4o-mini", n=1, structured_output=False, confidence=False, logprobs=True)[0]
    o = parse_output_translation("Sardinian", choice.message.content)
    messages.append(choice.message.content)
    preds.append(o["answer"])
    gold.append(inst["answer"])

chrf = torchmetrics.CHRFScore(return_sentence_level_score=True)
chrf(preds, gold)
score = chrf.compute()[1].mean()

English: A person who weighs 200 pounds (90kg) on Earth would weigh about 36 pounds (16kg) on Io. So the gravity, of course, pulls less on you.
Sardinian: Una persona chi pesat 200 pounds (90kg) in Terra pesat circa 36 pounds (16kg) in Io. Allora, sa gravità, naturalmente, at più de pocu pesa a tì.

___
English: In 1981, Vanda Miss Joaquim, an orchid hybrid, was chosen as the nation's national flower.
Sardinian: In 1981, Vanda Miss Joaquim, una ibrida di orchidea, est stata eletta comente su fioru nazionali de sa nazione.

___
English: Sikhs consider their faith to be a separate religion from Hinduism though they acknowledge its Hindu roots and traditions.
Sardinian: Sardinian: Sos Sikhs considerant sa loro fede comente una religione separada dae s'Hinduismu, puru chi riconoschint sos radichis e sas traditziones hindus.

___
English: Two beams of light have been rigged up to point skywards overnight.
Sardinian: Sardinian: Due fasci de luma si nd'hant fittados a puntare in su chèrrere i

100%|██████████| 200/200 [07:17<00:00,  2.19s/it]
/Users/zhengyaogu/miniconda3/envs/in-context-ssl/lib/python3.12/site-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `CHRFScore` from `torchmetrics` was deprecated and will be removed in 2.0. Import `CHRFScore` from `torchmetrics.text` instead.
  _future_warning(


In [34]:
score

tensor(0.3846)

## Naive-SemiICL

In [12]:
preds = []
gold = []
confidences = []
messages = []

new_ds_verbalized = []
new_ds_entropy = []

ds = TranslationDatasetBem()

k=0
for inst in tqdm(ds.train_iter(
    "in_context_ssl/reasoning/data/flores_bem_train.hf",
    k=k, answer=True, rationale=False, seed=42
)):
    choices = query_openai(client, inst["query"], n=1, model="gpt-4o-mini", structured_output=False, confidence=True, logprobs=True)

    o_verbalized = aggregate(choices, parser=lambda x: parse_output_translation("Bemba", x), confidence="verbalized", rationale=False)
    o_entropy = aggregate(choices, parser=lambda x: parse_output_translation("Bemba", x), confidence="entropy", rationale=False)

    d_verbalized = {
        "question": inst["question"],
        "answer": o_verbalized["answer"],
        "group": inst["group"],
        "confidence": o_verbalized["confidence"],
    }
    d_entropy = dict(d_verbalized)
    d_entropy["confidence"] = o_entropy["confidence"]
    new_ds_verbalized.append(d_verbalized)
    new_ds_entropy.append(d_entropy)

datasets.Dataset.from_pandas(pd.DataFrame(
    new_ds_verbalized
)).save_to_disk("in_context_ssl/reasoning/data/flores_bem_psl_k={}_verbalized.hf".format(k))
datasets.Dataset.from_pandas(pd.DataFrame(
    new_ds_entropy
)).save_to_disk("in_context_ssl/reasoning/data/flores_bem_psl_k={}_entropy.hf".format(k))

0it [00:00, ?it/s]

997it [26:49,  1.61s/it]
Saving the dataset (1/1 shards): 100%|██████████| 997/997 [00:00<00:00, 393907.41 examples/s]


In [ ]:
preds = []
gold = []
confidences = []
messages = []

new_ds_verbalized = []
new_ds_entropy = []

ds = TranslationDatasetFij()

k=0
for inst in tqdm(ds.train_iter(
    "in_context_ssl/reasoning/data/flores_fij_train.hf",
    k=k, answer=True, rationale=False, seed=42
)):
    choices = query_openai(client, inst["query"], n=1, model="gpt-4o-mini", structured_output=False, confidence=True, logprobs=True)

    o_verbalized = aggregate(choices, parser=lambda x: parse_output_translation("Fijian", x), confidence="verbalized", rationale=False)
    o_entropy = aggregate(choices, parser=lambda x: parse_output_translation("Fijian", x), confidence="entropy", rationale=False)

    d_verbalized = {
        "question": inst["question"],
        "answer": o_verbalized["answer"],
        "group": inst["group"],
        "confidence": o_verbalized["confidence"],
    }
    d_entropy = dict(d_verbalized)
    d_entropy["confidence"] = o_entropy["confidence"]
    new_ds_verbalized.append(d_verbalized)
    new_ds_entropy.append(d_entropy)

datasets.Dataset.from_pandas(pd.DataFrame(
    new_ds_verbalized
)).save_to_disk("in_context_ssl/reasoning/data/flores_fij_psl_k={}_verbalized.hf".format(k))
datasets.Dataset.from_pandas(pd.DataFrame(
    new_ds_entropy
)).save_to_disk("in_context_ssl/reasoning/data/flores_fij_psl_k={}_entropy.hf".format(k))

997it [27:45,  1.67s/it]
Saving the dataset (1/1 shards): 100%|██████████| 997/997 [00:00<00:00, 526930.58 examples/s]


In [ ]:
preds = []
gold = []
confidences = []
messages = []

new_ds_verbalized = []
new_ds_entropy = []

ds = TranslationDatasetSrd()

k=0
for inst in tqdm(ds.train_iter(
    "in_context_ssl/reasoning/data/flores_srd_train.hf",
    k=k, answer=True, rationale=False, seed=42
)):
    choices = query_openai(client, inst["query"], n=1, model="gpt-4o-mini", structured_output=False, confidence=True, logprobs=True)

    o_verbalized = aggregate(choices, parser=lambda x: parse_output_translation("Sardinian", x), confidence="verbalized", rationale=False)
    o_entropy = aggregate(choices, parser=lambda x: parse_output_translation("Sardinian", x), confidence="entropy", rationale=False)

    d_verbalized = {
        "question": inst["question"],
        "answer": o_verbalized["answer"],
        "group": inst["group"],
        "confidence": o_verbalized["confidence"],
    }
    d_entropy = dict(d_verbalized)
    d_entropy["confidence"] = o_entropy["confidence"]
    new_ds_verbalized.append(d_verbalized)
    new_ds_entropy.append(d_entropy)

datasets.Dataset.from_pandas(pd.DataFrame(
    new_ds_verbalized
)).save_to_disk("in_context_ssl/reasoning/data/flores_srd_psl_k={}_verbalized.hf".format(k))
datasets.Dataset.from_pandas(pd.DataFrame(
    new_ds_entropy
)).save_to_disk("in_context_ssl/reasoning/data/flores_srd_psl_k={}_entropy.hf".format(k))

997it [27:20,  1.65s/it]
Saving the dataset (1/1 shards): 100%|██████████| 997/997 [00:00<00:00, 403835.93 examples/s]


In [ ]:
preds = []
gold = []
confidences = []
messages = []

new_ds_verbalized = []
new_ds_entropy = []

ds = TranslationDatasetTyv()

k=0
for inst in tqdm(ds.train_iter(
    "in_context_ssl/reasoning/data/flores_tyv_train.hf",
    k=k, answer=True, rationale=False, seed=42
)):
    choices = query_openai(client, inst["query"], n=1, model="gpt-4o-mini", structured_output=False, confidence=True, logprobs=True)

    o_verbalized = aggregate(choices, parser=lambda x: parse_output_translation("Tuvan", x), confidence="verbalized", rationale=False)
    o_entropy = aggregate(choices, parser=lambda x: parse_output_translation("Tuvan", x), confidence="entropy", rationale=False)

    d_verbalized = {
        "question": inst["question"],
        "answer": o_verbalized["answer"],
        "group": inst["group"],
        "confidence": o_verbalized["confidence"],
    }
    d_entropy = dict(d_verbalized)
    d_entropy["confidence"] = o_entropy["confidence"]
    new_ds_verbalized.append(d_verbalized)
    new_ds_entropy.append(d_entropy)

datasets.Dataset.from_pandas(pd.DataFrame(
    new_ds_verbalized
)).save_to_disk("in_context_ssl/reasoning/data/flores_tyv_psl_k={}_verbalized.hf".format(k))
datasets.Dataset.from_pandas(pd.DataFrame(
    new_ds_entropy
)).save_to_disk("in_context_ssl/reasoning/data/flores_tyv_psl_k={}_entropy.hf".format(k))

997it [32:37,  1.96s/it]
Saving the dataset (1/1 shards): 100%|██████████| 997/997 [00:00<00:00, 356710.83 examples/s]


In [ ]:
preds = []
gold = []
confidences = []
messages = []

new_ds_verbalized = []
new_ds_entropy = []

ds = TranslationDatasetVec()

k=0
for inst in tqdm(ds.train_iter(
    "in_context_ssl/reasoning/data/flores_vec_train.hf",
    k=k, answer=True, rationale=False, seed=42
)):
    choices = query_openai(client, inst["query"], n=1, model="gpt-4o-mini", structured_output=False, confidence=True, logprobs=True)

    o_verbalized = aggregate(choices, parser=lambda x: parse_output_translation("Venetian", x), confidence="verbalized", rationale=False)
    o_entropy = aggregate(choices, parser=lambda x: parse_output_translation("Venetian", x), confidence="entropy", rationale=False)

    d_verbalized = {
        "question": inst["question"],
        "answer": o_verbalized["answer"],
        "group": inst["group"],
        "confidence": o_verbalized["confidence"],
    }
    d_entropy = dict(d_verbalized)
    d_entropy["confidence"] = o_entropy["confidence"]
    new_ds_verbalized.append(d_verbalized)
    new_ds_entropy.append(d_entropy)

datasets.Dataset.from_pandas(pd.DataFrame(
    new_ds_verbalized
)).save_to_disk("in_context_ssl/reasoning/data/flores_vec_psl_k={}_verbalized.hf".format(k))
datasets.Dataset.from_pandas(pd.DataFrame(
    new_ds_entropy
)).save_to_disk("in_context_ssl/reasoning/data/flores_vec_psl_k={}_entropy.hf".format(k))

997it [25:26,  1.53s/it]
Saving the dataset (1/1 shards): 100%|██████████| 997/997 [00:00<00:00, 409450.81 examples/s]
